# * Ex-T Revenue & Subs

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

pd.set_option('future.no_silent_downcasting', True)

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Actual : Monitoring
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [2]:
''' Execute Summany Data '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/ 
        SUBSTR(TM_KEY_DAY,1,6) AS TM_KEY_MTH, REMARK, METRIC_CD, METRIC_NAME
        , MAX(LOAD_DATE) LOAD_DATE, MIN(TM_KEY_DAY) AS START_DAY, MAX(TM_KEY_DAY) AS END_DAY
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
        , COUNT(DISTINCT AREA_TYPE) CNT_AREA_TYPE, COUNT(DISTINCT AREA_CD) CNT_AREA_CD, COUNT(1) ROW_CNT
    FROM (
        SELECT TM_KEY_DAY
            , CASE 	WHEN METRIC_CD LIKE 'TB%S%' 
                    THEN ROW_NUMBER() OVER (PARTITION BY SUBSTR(TM_KEY_DAY,1,6), METRIC_CD, AREA_CD ORDER BY TM_KEY_DAY DESC)
                    ELSE 1 END FLAG
            , CASE WHEN METRIC_CD LIKE 'TB%S%' THEN 'SUB' ELSE 'REV' END AGG_TYPE
            , METRIC_CD, METRIC_NAME, ACTUAL, COMP_CD, VERSION, AREA_CD, AREA_DESC, AREA_TYPE, LOAD_DATE, REMARK, SRC
        FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
        WHERE REMARK IN ('salisa', 'jutar11')
        AND NOT REGEXP_LIKE(METRIC_CD, '^TCAP|CORP$')
        AND EXISTS (
            SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
            FROM (
                SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
                UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            ) TMP
            WHERE TMP_CD = AREA_CD )
        AND TM_KEY_DAY >= {v_start_date} 
    ) TMP
    WHERE FLAG = 1
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, SUBSTR(TM_KEY_DAY,1,6)
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Summany DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Summany DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20240101

TDMDBPR : Connected

   -> Execute query... 2025-04-02, 10:02:27

   -> Summany DataFrame : 427 rows, 17 columns

TDMDBPR : Disconnected


In [3]:
''' Current Month Revenue '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_overall_df = chk_src_df.copy()
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['TM_KEY_MTH']==curr_mth]
src_t_rev_overall_df = src_t_rev_overall_df.loc[src_t_rev_overall_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_overall_df = src_t_rev_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_overall_df = src_t_rev_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_overall_df[col] = src_t_rev_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-04-01 04:45:29,20250301,20250329,0,0,"2,330,176,528","2,330,165,110","2,330,165,110","2,320,480,127",0,5,1104,32016
1,TB2R000100,Postpaid Revenue,2025-04-01 04:20:09,20250302,20250331,0,"3,688,757,063","3,688,589,279","3,688,587,485","3,688,587,485",0,0,5,177,1770
2,TB2R010100,Postpaid Revenue B2C,2025-04-01 04:21:16,20250302,20250331,0,"3,323,197,818","3,323,114,268","3,323,112,474","3,323,112,474","3,312,385,680",0,6,1105,10532
3,TB2R020100,Postpaid Revenue B2B,2025-04-01 04:20:54,20250302,20250331,0,"365,559,245","365,475,010","365,475,010","365,475,010",0,0,5,177,1662
4,TB3R000100,Broadband Revenue,2025-04-01 05:09:23,20250302,20250328,0,0,"1,609,888,519","1,609,888,519","1,609,888,519","1,609,974,551",0,5,1102,9910
5,TB4R000100,TVS Revenue,2025-04-01 05:17:46,20250302,20250331,0,0,"207,090,230","207,153,092","207,119,120",0,0,4,176,1760


In [4]:
''' Average Revenue per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
rev_metric_list = ['TB1R000100', 'TB2R000100', 'TB2R010100', 'TB2R020100', 'TB3R000100', 'TB4R000100']

src_t_rev_avg_mth_df = chk_src_df.copy()
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.loc[src_t_rev_avg_mth_df['METRIC_CD'].isin(rev_metric_list)]
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_rev_avg_mth_df = src_t_rev_avg_mth_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_rev_avg_mth_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_rev_avg_mth_df[col] = src_t_rev_avg_mth_df[col].apply(lambda x: format(x, ',.0f'))

src_t_rev_avg_mth_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1R000100,Prepay Revenue,2025-03-20 01:42:42,20240101,20250228,0,0,"2,331,116,542","2,331,084,640","2,331,084,640","2,321,354,989",0,5,1104,469200
1,TB2R000100,Postpaid Revenue,2025-03-19 11:39:10,20240102,20250228,0,"3,702,523,438","3,671,427,102","3,671,411,395","3,671,411,395",0,0,5,177,24603
2,TB2R010100,Postpaid Revenue B2C,2025-03-19 11:40:02,20240102,20250228,0,"3,293,619,861","3,287,217,821","3,287,203,341","3,287,204,311","3,262,416,481",0,6,1105,148232
3,TB2R020100,Postpaid Revenue B2B,2025-03-19 11:39:49,20240102,20250228,0,"408,903,578","384,209,282","384,208,054","384,208,054",0,0,5,177,24603
4,TB3R000100,Broadband Revenue,2025-03-21 14:11:16,20240102,20250228,0,0,"1,573,227,163","1,573,118,602","1,573,088,544","1,560,922,362",0,5,1102,138718
5,TB4R000100,TVS Revenue,2025-03-14 13:59:20,20240102,20250228,0,0,"328,867,343","323,172,106","327,520,964","322,352,854",0,5,1104,106301


In [5]:
''' Current Month Subscription '''

curr_mth = chk_src_df['TM_KEY_MTH'].max()
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH']==curr_mth]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'sum', 'P':'sum', 'G':'sum', 'H':'sum', 'HH':'sum', 'CCAA':'sum', 'CCAATT':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-04-01 04:51:06,20250329,20250329,0,0,"17,402,654","17,402,654","17,402,654","17,340,945",0,5,1104,1104
1,TB1S000600,Prepay Active Caller Subs,2025-03-16 04:50:51,20250314,20250314,0,"14,567,698","14,355,888","14,355,888","14,355,888","14,304,384",0,6,1105,1105
2,TB2S000500,Postpaid Active Subs,2025-04-01 05:13:36,20250331,20250331,0,"10,201,287","10,200,756","10,200,744","10,200,744","10,125,910",0,6,1105,1105
3,TB2S010500,Postpaid Active Subs B2C,2025-04-01 05:14:31,20250331,20250331,0,"9,016,036","9,015,793","9,015,781","9,015,781","8,960,676",0,6,1105,1105
4,TB2S020400,Postpaid Active Subs B2B,2025-04-01 05:12:40,20250331,20250331,0,"1,185,251","1,184,963","1,184,963","1,184,963",0,0,5,177,177
5,TB3S000500,Broadband Active Subs,2025-04-01 04:56:01,20250331,20250331,0,0,"3,127,140","3,127,140","3,127,140","3,127,140",0,5,1102,1102
6,TB4S000500,TVS Active Subs,2025-04-01 04:56:26,20250331,20250331,0,0,"1,153,137","1,153,137","1,153,137","1,153,122",0,5,1104,1104


In [6]:
''' Average Subscription per Month '''

avg_range_start = '202401'
avg_range_end = '202502'
sub_metric_list = ['TB1S000500', 'TB1S000600', 'TB2S000500', 'TB2S010500', 'TB2S020400', 'TB3S000500', 'TB4S000500']

src_t_sub_overall_df = chk_src_df.copy()
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['TM_KEY_MTH'].between(avg_range_start, avg_range_end)]
src_t_sub_overall_df = src_t_sub_overall_df.loc[src_t_sub_overall_df['METRIC_CD'].isin(sub_metric_list)]
src_t_sub_overall_df = src_t_sub_overall_df.groupby(['METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'C':'mean', 'P':'mean', 'G':'mean', 'H':'mean', 'HH':'mean', 'CCAA':'mean', 'CCAATT':'mean', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
src_t_sub_overall_df = src_t_sub_overall_df.fillna(0).sort_values(by=['METRIC_CD']).reset_index()

mod_col_list = src_t_sub_overall_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    src_t_sub_overall_df[col] = src_t_sub_overall_df[col].apply(lambda x: format(x, ',.0f'))

src_t_sub_overall_df

,METRIC_CD,METRIC_NAME,LOAD_DATE,START_DAY,END_DAY,C,P,G,H,HH,CCAA,CCAATT,CNT_AREA_TYPE,CNT_AREA_CD,ROW_CNT
0,TB1S000500,Prepay Usage Subs,2025-03-20 12:12:14,20240131,20250228,0,0,"17,662,374","17,662,181","17,662,181","17,600,486",0,5,1104,15456
1,TB1S000600,Prepay Active Caller Subs,2025-03-10 04:51:05,20240131,20250228,0,"14,441,024","14,342,605","14,362,024","14,362,051","14,312,460",0,6,1105,15470
2,TB2S000500,Postpaid Active Subs,2025-03-19 21:31:43,20240131,20250228,0,"10,021,687","9,921,762","9,922,029","9,922,053","9,849,626",0,6,1105,15470
3,TB2S010500,Postpaid Active Subs B2C,2025-03-19 21:32:10,20240131,20250228,0,"8,891,597","8,887,629","8,887,839","8,887,863","8,832,683",0,6,1105,15470
4,TB2S020400,Postpaid Active Subs B2B,2025-03-19 21:31:15,20240131,20250228,0,"1,130,090","1,034,133","1,034,190","1,034,190",0,0,5,177,2478
5,TB3S000500,Broadband Active Subs,2025-03-01 04:55:28,20240131,20250228,0,0,"3,053,895","3,055,196","3,055,196","3,055,841",0,5,1102,15412
6,TB4S000500,TVS Active Subs,2025-03-01 04:55:38,20240131,20250228,0,0,"1,220,146","1,219,989","1,219,989","1,307,223",0,5,8408,22760


In [7]:
# ''' METRIC Summary '''

# src_t_rev_mth_df = chk_src_df.copy()
# src_t_rev_mth_df = src_t_rev_mth_df.groupby(['REMARK', 'METRIC_CD', 'METRIC_NAME']).agg({'LOAD_DATE':'max', 'START_DAY':'min', 'END_DAY':'max', 'P':'sum', 'CNT_AREA_TYPE':'max', 'CNT_AREA_CD':'max', 'ROW_CNT':'sum'})
# src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD']).reset_index()

# mod_col_list = src_t_rev_mth_df.iloc[:, 6:7].columns.tolist()
# for col in mod_col_list:
#     src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_rev_mth_df.to_string(max_cols=10)}')

In [8]:
# ''' METRIC Current Month '''

# # v_tm_key_mth = '202502'
# curr_mth = chk_src_df['TM_KEY_MTH'].max()
# src_t_rev_mth_df_cols = ['REMARK', 'METRIC_CD', 'METRIC_NAME', 'LOAD_DATE', 'TM_KEY_MTH', 'START_DAY', 'END_DAY', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT']
# print(f'\nParameter input...\n')
# print(f'   -> v_tm_key_mth: {curr_mth}')

# src_t_rev_mth_df = chk_src_df[src_t_rev_mth_df_cols].copy()
# src_t_rev_mth_df = src_t_rev_mth_df.loc[src_t_rev_mth_df['TM_KEY_MTH']==curr_mth]
# src_t_rev_mth_df = src_t_rev_mth_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_MTH']).reset_index(drop=True)

# mod_col_list = src_t_rev_mth_df.iloc[:, 7:14].columns.tolist()
# for col in mod_col_list:
#     src_t_rev_mth_df[col] = src_t_rev_mth_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{src_t_rev_mth_df.to_string(max_cols=17)}')

## Actual : Transaction
->  FCT_KPI_NEWCO_PERF_ACTUAL

In [9]:
''' Execute Daily by case '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
print(f'\n{TDMDBPR_db} : Connected')
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+ PARALLEL(8)*/ 
         REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
        , MAX(LOAD_DATE) LOAD_DATE
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL END) HH
        , SUM(CASE WHEN AREA_TYPE = 'CCAA' THEN ACTUAL END) CCAA
        , SUM(CASE WHEN AREA_TYPE = 'CCAATT' THEN ACTUAL END) CCAATT
    FROM GEOSPCAPPO.FCT_KPI_NEWCO_PERF_ACTUAL
    --WHERE METRIC_CD = 'TB3R000100' --TOL Revenue
    WHERE METRIC_CD IN (
        'TB1R000100' --'Prepay Revenue
        , 'TB2R010100' --Postpaid Revenue B2C
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
         )
    AND EXISTS (
        SELECT 1 -->> AREA_CD IN (P, G, H, R, HH, CCAA, CCAATT)
        FROM (
            SELECT DISTINCT ORGID_P AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_G AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_R AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_H AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT ORGID_HH AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT SUBSTR(CCAATT,1,4) AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
            UNION SELECT DISTINCT CCAATT AS TMP_CD FROM CDSAPPO.DIM_MOOC_AREA NOLOCK WHERE REMARK <> 'Dummy' AND TEAM_CODE <> 'ไม่ระบุ'
        ) TMP
        WHERE TMP_CD = AREA_CD )
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY REMARK, METRIC_CD, METRIC_NAME, TM_KEY_DAY
""")


try:
    execute_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
    print(f'\n   -> Execute query... {execute_datetime}')

    # Get : Daily DataFrame
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])
    print(f'\n   -> Daily DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()
    print(f'\n{TDMDBPR_db} : Disconnected')


Parameter input...

   -> v_start_date: 20250101



TDMDBPR : Connected

   -> Execute query... 2025-04-02, 10:04:09

   -> Daily DataFrame : 173 rows, 12 columns

TDMDBPR : Disconnected


In [10]:
''' METRIC by Day '''

v_tm_key_day = 20250101
# src_t_rev_day_df = ['SRC', 'REMARK', 'TM_KEY_DAY', 'METRIC_CD', 'METRIC_NAME', 'C', 'P', 'G', 'H', 'HH', 'CCAA', 'CCAATT', 'LOAD_DATE']
print(f'\nParameter input...\n')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

# src_t_rev_day_df = chk_src_df[src_t_rev_day_df_cols].copy()
src_t_rev_day_df = chk_src_df.copy()
src_t_rev_day_df = src_t_rev_day_df.loc[src_t_rev_day_df['TM_KEY_DAY']>=v_tm_key_day]
src_t_rev_day_df = src_t_rev_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = src_t_rev_day_df.iloc[:, 4:11].columns.tolist()
for col in mod_col_list:
    src_t_rev_day_df[col] = src_t_rev_day_df[col].apply(lambda x: format(x, ',.0f'))

print(f'\nCreate : src_t_rev_day_df...')
print(f'\n   -> DataFrame : {chk_src_df.shape[0]} rows, {chk_src_df.shape[1]} columns')
# print(f'\n{src_t_rev_day_df.to_string(max_cols=17)}')


Parameter input...

   -> v_tm_key_day: 20250101

Create : src_t_rev_day_df...

   -> DataFrame : 173 rows, 12 columns


In [11]:
''' TB1R000100 : Prepay Revenue '''

v_metric_cd = 'TB1R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB1R000100 : Prepay Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB1R000100_day_df = chk_src_df.copy()
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['METRIC_CD']==v_metric_cd]
TB1R000100_day_df = TB1R000100_day_df.loc[TB1R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB1R000100_day_df = TB1R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB1R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB1R000100_day_df[col] = TB1R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB1R000100_day_df


TB1R000100 : Prepay Revenue...

   -> v_metric_cd: TB1R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB1R000100,Prepay Revenue,20250301,2025-04-01 04:45:29,0,0,"98,892,917","98,891,932","98,891,932","98,433,031",0
1,jutar11,TB1R000100,Prepay Revenue,20250302,2025-04-01 04:45:29,0,0,"97,914,661","97,914,420","97,914,420","97,447,900",0
2,jutar11,TB1R000100,Prepay Revenue,20250303,2025-04-01 04:45:29,0,0,"97,342,738","97,342,212","97,342,212","96,907,806",0
3,jutar11,TB1R000100,Prepay Revenue,20250304,2025-04-01 04:45:29,0,0,"87,088,235","87,087,757","87,087,757","86,698,348",0
4,jutar11,TB1R000100,Prepay Revenue,20250305,2025-04-01 04:45:29,0,0,"88,603,331","88,603,174","88,603,174","88,248,067",0
5,jutar11,TB1R000100,Prepay Revenue,20250306,2025-04-01 04:45:29,0,0,"83,017,557","83,017,431","83,017,431","82,670,415",0
6,jutar11,TB1R000100,Prepay Revenue,20250307,2025-04-01 04:45:29,0,0,"84,170,500","84,169,267","84,169,267","83,822,190",0
7,jutar11,TB1R000100,Prepay Revenue,20250308,2025-04-01 04:45:29,0,0,"79,925,578","79,925,496","79,925,496","79,593,003",0
8,jutar11,TB1R000100,Prepay Revenue,20250309,2025-04-01 04:45:29,0,0,"77,199,904","77,199,854","77,199,854","76,885,717",0
9,jutar11,TB1R000100,Prepay Revenue,20250310,2025-04-01 04:45:29,0,0,"83,602,278","83,602,161","83,602,161","83,264,748",0


In [12]:
''' TB2R010100 : Postpaid Revenue B2C '''

v_metric_cd = 'TB2R010100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB2R010100 : Postpaid Revenue B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB2R010100_day_df = chk_src_df.copy()
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['METRIC_CD']==v_metric_cd]
TB2R010100_day_df = TB2R010100_day_df.loc[TB2R010100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB2R010100_day_df = TB2R010100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB2R010100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB2R010100_day_df[col] = TB2R010100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB2R010100_day_df


TB2R010100 : Postpaid Revenue B2C...

   -> v_metric_cd: TB2R010100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,jutar11,TB2R010100,Postpaid Revenue B2C,20250302,2025-04-01 04:21:16,0,"475,681,338","475,672,925","475,672,026","475,672,026","474,153,424",0
1,jutar11,TB2R010100,Postpaid Revenue B2C,20250306,2025-04-01 04:21:16,0,"280,068,025","280,062,103","280,062,103","280,062,103","278,758,063",0
2,jutar11,TB2R010100,Postpaid Revenue B2C,20250310,2025-04-01 04:21:16,0,"442,370,575","442,361,497","442,361,497","442,361,497","441,084,572",0
3,jutar11,TB2R010100,Postpaid Revenue B2C,20250313,2025-04-01 04:21:16,0,"389,324,834","389,315,042","389,314,593","389,314,593","388,173,082",0
4,jutar11,TB2R010100,Postpaid Revenue B2C,20250316,2025-04-01 04:21:16,0,"353,872,182","353,866,139","353,865,733","353,865,733","352,766,207",0
5,jutar11,TB2R010100,Postpaid Revenue B2C,20250319,2025-04-01 04:21:16,0,"352,201,541","352,192,667","352,192,667","352,192,667","351,027,035",0
6,jutar11,TB2R010100,Postpaid Revenue B2C,20250322,2025-04-01 04:21:16,0,"360,531,872","360,524,671","360,524,631","360,524,631","359,344,272",0
7,jutar11,TB2R010100,Postpaid Revenue B2C,20250325,2025-04-01 04:21:16,0,"326,527,862","326,515,425","326,515,425","326,515,425","325,530,840",0
8,jutar11,TB2R010100,Postpaid Revenue B2C,20250328,2025-04-01 04:21:16,0,"342,520,751","342,504,962","342,504,962","342,504,962","341,449,753",0
9,jutar11,TB2R010100,Postpaid Revenue B2C,20250331,2025-04-01 04:21:16,0,"98,838","98,838","98,838","98,838","98,432",0


In [13]:
''' TB3R000100 : Broadband Revenue '''

v_metric_cd = 'TB3R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB3R000100 : Broadband Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3R000100_day_df = chk_src_df.copy()
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['METRIC_CD']==v_metric_cd]
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB3R000100_day_df = TB3R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB3R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB3R000100_day_df[col] = TB3R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3R000100_day_df


TB3R000100 : Broadband Revenue...

   -> v_metric_cd: TB3R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB3R000100,Broadband Revenue,20250302,2025-04-01 05:09:23,0,0,"217,468,998","217,468,998","217,468,998","217,504,335",0
1,salisa,TB3R000100,Broadband Revenue,20250306,2025-04-01 05:09:23,0,0,"142,949,499","142,949,499","142,949,499","142,949,499",0
2,salisa,TB3R000100,Broadband Revenue,20250310,2025-04-01 05:09:23,0,0,"232,319,883","232,319,883","232,319,883","232,345,045",0
3,salisa,TB3R000100,Broadband Revenue,20250313,2025-04-01 05:09:23,0,0,"172,672,394","172,672,394","172,672,394","172,672,394",0
4,salisa,TB3R000100,Broadband Revenue,20250316,2025-04-01 05:09:23,0,0,"173,663,311","173,663,311","173,663,311","173,666,647",0
5,salisa,TB3R000100,Broadband Revenue,20250319,2025-04-01 05:09:23,0,0,"166,393,342","166,393,342","166,393,342","166,393,874",0
6,salisa,TB3R000100,Broadband Revenue,20250322,2025-04-01 05:09:23,0,0,"166,820,109","166,820,109","166,820,109","166,824,681",0
7,salisa,TB3R000100,Broadband Revenue,20250325,2025-04-01 05:09:23,0,0,"165,420,384","165,420,384","165,420,384","165,420,384",0
8,salisa,TB3R000100,Broadband Revenue,20250328,2025-04-01 05:09:23,0,0,"172,180,599","172,180,599","172,180,599","172,197,692",0


In [14]:
''' TB4R000100 : TVS Revenue '''

v_metric_cd = 'TB4R000100'
v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250201
print(f'\nTB4R000100 : TVS Revenue...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4R000100_day_df = chk_src_df.copy()
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['METRIC_CD']==v_metric_cd]
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB4R000100_day_df = TB4R000100_day_df.fillna(0).sort_values(by=['REMARK', 'METRIC_CD', 'TM_KEY_DAY']).reset_index(drop=True)

mod_col_list = TB4R000100_day_df.iloc[:, 5:12].columns.tolist()
for col in mod_col_list:
    TB4R000100_day_df[col] = TB4R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4R000100_day_df


TB4R000100 : TVS Revenue...

   -> v_metric_cd: TB4R000100
   -> v_tm_key_day: 20250301


,REMARK,METRIC_CD,METRIC_NAME,TM_KEY_DAY,LOAD_DATE,C,P,G,H,HH,CCAA,CCAATT
0,salisa,TB4R000100,TVS Revenue,20250302,2025-04-01 05:17:46,0,0,"34,087,365","34,079,706","34,091,178",0,0
1,salisa,TB4R000100,TVS Revenue,20250306,2025-04-01 05:17:46,0,0,"9,580,155","9,659,093","9,654,980",0,0
2,salisa,TB4R000100,TVS Revenue,20250310,2025-04-01 05:17:46,0,0,"29,692,347","29,776,686","29,789,708",0,0
3,salisa,TB4R000100,TVS Revenue,20250313,2025-04-01 05:17:46,0,0,"21,811,722","21,858,042","21,855,353",0,0
4,salisa,TB4R000100,TVS Revenue,20250316,2025-04-01 05:17:46,0,0,"23,606,279","23,623,490","23,617,050",0,0
5,salisa,TB4R000100,TVS Revenue,20250319,2025-04-01 05:17:46,0,0,"20,529,197","20,541,381","20,541,285",0,0
6,salisa,TB4R000100,TVS Revenue,20250322,2025-04-01 05:17:46,0,0,"20,877,853","20,884,053","20,876,410",0,0
7,salisa,TB4R000100,TVS Revenue,20250325,2025-04-01 05:17:46,0,0,"21,263,644","21,263,829","21,259,397",0,0
8,salisa,TB4R000100,TVS Revenue,20250328,2025-04-01 05:17:46,0,0,"23,044,903","23,101,998","23,109,863",0,0
9,salisa,TB4R000100,TVS Revenue,20250331,2025-04-01 05:17:46,0,0,"2,596,765","2,364,815","2,323,898",0,0
